In [1]:
import urllib
import pandas as pd
import numpy as np
import json
import requests
import time

In [32]:
disease_mod = pd.read_csv("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Health AI Tracking\\disease_modality\\disease_modality_raw.csv", encoding="latin1", index_col = 0)
disease_mod.info()
disease_mod.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28703 entries, 0 to 28702
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pmid             28703 non-null  int64  
 1   title            28702 non-null  object 
 2   abstract         28703 non-null  object 
 3   disease_answer   28703 non-null  object 
 4   disease_score    28703 non-null  float64
 5   modality_answer  28703 non-null  object 
 6   modality_score   28703 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 1.8+ MB


,pmid,title,abstract,disease_answer,disease_score,modality_answer,modality_score
0,35309968,mTeeth: Identifying Brushing Teeth Surfaces Us...,Ensuring that all the teeth surfaces are adequ...,oral diseases,0.304024,inertial sensors,0.559315
1,35330785,Development of a Machine Learning Algorithm fo...,Reverse total shoulder arthroplasty (rTSA) off...,Reverse total shoulder arthroplasty,0.377661,Office of Statewide Health Planning and Develo...,0.363589
2,35330977,ECG Restitution Analysis and Machine Learning ...,Atrial fibrillation is the most frequent arrhy...,paroxysmal atrial fibrillation,0.391101,normal sinus-rhythm ECGs,0.316086
3,35330920,Segmenting Thoracic Cavities with Neoplastic L...,Automatic segmentation of thoracic cavity stru...,neoplastic disease,0.441814,CT images,0.502925
4,30814403,Combined machine learning and functional magne...,Hypoxia exposure during high-altitude expediti...,psychomotor impairment,0.347314,Rs-fMRI,0.341074
5,33075216,Simple risk score to screen for prediabetes: A...,The progression from prediabetes to typeÃÂ 2 ...,prediabetes,0.198258,Qatar Biobank cohort,0.448133
6,34933537,The use and applicability of machine learning ...,Artificial intelligence (AI) is increasingly u...,benign prostatic enlargement,0.648665,Clinical data,0.333634
7,34897147,Automation of a Rule-based Workflow to Estimat...,Myelination-related MR signal changes in white...,Sturge-Weber syndrome,0.564898,T1-weighted images (T1WIs) and T2-weighted ima...,0.187111
8,34176867,Deep Learning Algorithm to Detect Cardiac Sarc...,Because the early diagnosis of subclinical car...,cardiac sarcoidosis,0.436207,echocardiographic movies,0.345082
9,25296360,Estimating the average length of hospitalizati...,Exposure to air pollutants is associated with ...,pneumonia,0.449299,air pollutants,0.167810


In [24]:
medcat_info = requests.get("http://localhost:5000/api/info")
print(medcat_info)
medcat_info.json()

<Response [200]>


{'service_app_name': 'MedCAT',
 'service_language': 'en',
 'service_version': '1.4.0',
 'service_model': 'MedMen'}

In [ ]:
#load API responses from previously analysed data
#with open("disease_list", "r") as fp:
#    disease_list = json.load(fp)
#
#print(disease_list[0])

In [4]:
post_URL = "http://localhost:5000/api/process"
disease_list = []
count = 0

#post requests to API and create list of lists including matched phrase/cui/type for each disease_answer
for item in disease_mod["disease_answer"]:
    formatted_data = {"content" : {"text" : str(item)}}
    response = requests.post(post_URL, json = formatted_data)
    response_json = response.json()
    response_data = response_json["result"]["annotations"][0]
    compiled_list = []
    if len(response_data) > 0:
        for item in list(response_data.values()):
            data_list = []
            data_list.append(item["pretty_name"])
            data_list.append(item["cui"])
            data_list.append(item["types"][0])
            compiled_list.append(data_list)
    else:
        compiled_list = []
    disease_list.append(compiled_list)
    count += 1
    if count%1000 == 0:
        print("Processed: "+str(count)+" records")

Processed: 1000 records
Processed: 2000 records
Processed: 3000 records
Processed: 4000 records
Processed: 5000 records
Processed: 6000 records
Processed: 7000 records
Processed: 8000 records
Processed: 9000 records
Processed: 10000 records
Processed: 11000 records
Processed: 12000 records
Processed: 13000 records
Processed: 14000 records
Processed: 15000 records
Processed: 16000 records
Processed: 17000 records
Processed: 18000 records
Processed: 19000 records
Processed: 20000 records
Processed: 21000 records
Processed: 22000 records
Processed: 23000 records
Processed: 24000 records
Processed: 25000 records
Processed: 26000 records
Processed: 27000 records
Processed: 28000 records


In [27]:
disease_list_final = []
location_list_final = []
finding_list_final = []
combined_list_final = []
disease_identifiers = ['Acquired Abnormality', 'Anatomical Abnormality', 'Bacterium', 'Cell or Molecular Dysfunction', 'Congenital Abnormality', 'Disease or Syndrome', 'Eukaryote', 'Fungus', 'Injury or Poisoning', 'Mental Process', 'Mental or Behavioral Dysfunction', 'Neoplastic Process', 'Organ or Tissue Function', 'Pathologic Function', 'Sign or Symptom', 'Therapeutic or Preventive Procedure', 'Virus']
location_identifiers = ['Body Location or Region', 'Body Part, Organ, or Organ Component', 'Body Space or Junction', 'Body Substance', 'Body System', 'Spatial Concept', 'Tissue']
finding_identifiers = ['Acquired Abnormality', 'Anatomical Abnormality', 'Finding', 'Functional Concept']

#create list of only matched diseases as defined by type
for item in disease_list:
    item_list = []
    for x in item:
        #add concept if the type is within the identifiers and not already in the list for this paper
        if x[2] in disease_identifiers:
            if x not in item_list:
                item_list.append(x)
    disease_list_final.append(item_list)
    
#create list of only matched locations as defined by type
for item in disease_list:
    item_list = []
    for x in item:
        #add concept if the type is within the identifiers and not already in the list for this paper
        if x[2] in location_identifiers:
            if x not in item_list:
                item_list.append(x)
    location_list_final.append(item_list)
    
#create list of only matched findings as defined by type
for item in disease_list:
    item_list = []
    for x in item:
        #add concept if the type is within the identifiers and not already in the list for this paper
        if x[2] in finding_identifiers:
            if x not in item_list:
                item_list.append(x)
    finding_list_final.append(item_list)
    
#create list of combined concepts as defined by type
for item in disease_list:
    location = []
    finding = []
    combined = []
    for x in item:
        #add concept if the type is within the identifiers and not already in the list for this paper
        if x[2] in location_identifiers:
            if x not in location:
                location.append(x)
        #add concept if the type is within the identifiers and not already in the list for this paper
        if x[2] in finding_identifiers:
            if x not in finding:
                finding.append(x)
    #only add if one from each category is present
    if len(location) > 0 and len(finding) > 0:
        combined.extend(location)
        combined.extend(finding)
        combined_list_final.append(combined)
    else:
        combined_list_final.append([])

print(disease_list_final[0:10])
print(location_list_final[0:10])
print(finding_list_final[0:10])
print(combined_list_final[0:10])

[[['Disease, Mouth', 'C0026636', 'Disease or Syndrome']], [['Arthroplasties, Shoulder Replacement', 'C0186657', 'Therapeutic or Preventive Procedure']], [['Atrial Fibrillation', 'C0004238', 'Disease or Syndrome']], [['Neoplastic disease', 'C1882062', 'Neoplastic Process']], [], [['Prediabetes syndrome', 'C0362046', 'Disease or Syndrome']], [['Enlargement (morphologic abnormality)', 'C2711450', 'Anatomical Abnormality']], [], [['Cardiac sarcoidosis', 'C0392077', 'Disease or Syndrome']], [['Pneumonia', 'C0032285', 'Disease or Syndrome']]]
[[], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [['Impaired', 'C0221099', 'Functional Concept']], [], [['Enlargement (morphologic abnormality)', 'C2711450', 'Anatomical Abnormality']], [], [], []]
[[], [], [], [], [], [], [], [], [], []]


In [ ]:
#load API responses from previously analysed data
#with open("modality_list", "r") as fp:
#    modality_list = json.load(fp)
#    
#print(modality_list[0])

In [6]:
modality_list = []
count = 0

#post requests to API and create list of lists including matched phrase/cui/type for each modality_answer
for item in disease_mod["modality_answer"]:
    formatted_data = {"content" : {"text" : str(item)}}
    response = requests.post(post_URL, json = formatted_data)
    response_json = response.json()
    response_data = response_json["result"]["annotations"][0]
    compiled_list = []
    if len(response_data) > 0:
        for item in list(response_data.values()):
            data_list = []
            data_list.append(item["pretty_name"])
            data_list.append(item["cui"])
            data_list.append(item["types"][0])
            compiled_list.append(data_list)
    else:
        compiled_list = []
    modality_list.append(compiled_list)
    count += 1
    if count%1000 == 0:
        print("Processed: "+str(count)+" records")

Processed: 1000 records
Processed: 2000 records
Processed: 3000 records
Processed: 4000 records
Processed: 5000 records
Processed: 6000 records
Processed: 7000 records
Processed: 8000 records
Processed: 9000 records
Processed: 10000 records
Processed: 11000 records
Processed: 12000 records
Processed: 13000 records
Processed: 14000 records
Processed: 15000 records
Processed: 16000 records
Processed: 17000 records
Processed: 18000 records
Processed: 19000 records
Processed: 20000 records
Processed: 21000 records
Processed: 22000 records
Processed: 23000 records
Processed: 24000 records
Processed: 25000 records
Processed: 26000 records
Processed: 27000 records
Processed: 28000 records


In [39]:
modality_list_final = []
cm_final = []
modality_identifiers = ['Clinical Attribute', 'Diagnostic Procedure', 'Immunologic Factor', 'Intellectual Product', 'Laboratory Procedure', 'Laboratory or Test Result', 'Medical Device', 'Molecular Biology Research Technique', 'Molecular Function', 'Nucleic Acid, Nucleoside, or Nucleotide', 'Nucleotide Sequence', 'Organ or Tissue Function', 'Research Activity', 'Substance']

#create list of only matched modalities as defined by type
for item in modality_list:
    item_list = []
    for x in item:
        #add concept if the type is within the identifiers and not already in the list for this paper
        if x[2] in modality_identifiers:
            if x not in item_list:
                item_list.append(x)
    modality_list_final.append(item_list)
    
print(modality_list_final[0:10])

[[], [['Intellectual Product', 'C0282574', 'Intellectual Product']], [['Electrocardiography', 'C1623258', 'Diagnostic Procedure']], [['X-Ray Computed Tomography', 'C0040405', 'Diagnostic Procedure']], [], [], [['Clinical data', 'C1516606', 'Intellectual Product']], [['Diagnostic Imaging', 'C0011923', 'Diagnostic Procedure'], ['Study', 'C2603343', 'Research Activity'], ['simulation', 'C0679083', 'Research Activity'], ['Rule (guideline)', 'C0870077', 'Intellectual Product'], ['Image', 'C1704922', 'Intellectual Product'], ['Laboratory Procedures', 'C0022885', 'Laboratory Procedure'], ['Data Set', 'C0150098', 'Intellectual Product'], ['Magnetic Resonance Imaging', 'C0024485', 'Diagnostic Procedure'], ['cross-validation', 'C0681935', 'Research Activity'], ['Gestation Category Code - Full Term', 'C1547236', 'Intellectual Product'], ['Medical Image', 'C1704254', 'Intellectual Product']], [['Echocardiography', 'C0013516', 'Diagnostic Procedure']], []]


In [ ]:
#save API responses to file for later use to avoid lengthly analysis process
with open("disease_list", "w") as fp:
    json.dump(disease_list, fp)
    
with open("modality_list", "w") as fp:
    json.dump(modality_list, fp)

In [42]:
#add lists as series to dataframe
disease_mod["all_discovered_disease"] = pd.Series(disease_list)
disease_mod["discovered_disease"] = pd.Series(disease_list_final)
disease_mod["discovered_location"] = pd.Series(location_list_final)
disease_mod["discovered_finding"] = pd.Series(finding_list_final)
disease_mod["discovered_disease_combined"] = pd.Series(combined_list_final)
disease_mod["all_discovered_modality"] = pd.Series(modality_list)
disease_mod["discovered_modality"] = pd.Series(modality_list_final)

In [36]:
disease_mod = disease_mod.applymap(lambda x: np.NaN if x == [] else x)
disease_mod = disease_mod.applymap(lambda x: np.NaN if x == ["No match"] else x)
disease_mod.to_csv("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Health AI Tracking\\disease_modality\\disease_modality_discovered.csv")
disease_mod.info()
disease_mod.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28703 entries, 0 to 28702
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   pmid                         28703 non-null  int64  
 1   title                        28702 non-null  object 
 2   abstract                     28703 non-null  object 
 3   disease_answer               28703 non-null  object 
 4   disease_score                28703 non-null  float64
 5   modality_answer              28703 non-null  object 
 6   modality_score               28703 non-null  float64
 7   all_discovered_disease       27046 non-null  object 
 8   discovered_disease           21754 non-null  object 
 9   discovered_location          3105 non-null   object 
 10  discovered_finding           3330 non-null   object 
 11  discovered_disease_combined  827 non-null    object 
 12  all_discovered_modality      26347 non-null  object 
 13  discovered_modal

,pmid,title,abstract,disease_answer,disease_score,modality_answer,modality_score,all_discovered_disease,discovered_disease,discovered_location,discovered_finding,discovered_disease_combined,all_discovered_modality,discovered_modality
0,35309968,mTeeth: Identifying Brushing Teeth Surfaces Us...,Ensuring that all the teeth surfaces are adequ...,oral diseases,0.304024,inertial sensors,0.559315,"[[Disease, Mouth, C0026636, Disease or Syndrome]]","[[Disease, Mouth, C0026636, Disease or Syndrome]]",NaN,NaN,NaN,NaN,NaN
1,35330785,Development of a Machine Learning Algorithm fo...,Reverse total shoulder arthroplasty (rTSA) off...,Reverse total shoulder arthroplasty,0.377661,Office of Statewide Health Planning and Develo...,0.363589,"[[Arthroplasties, Shoulder Replacement, C01866...","[[Arthroplasties, Shoulder Replacement, C01866...",NaN,NaN,NaN,"[[Intellectual Product, C0282574, Intellectual...","[[Intellectual Product, C0282574, Intellectual..."
2,35330977,ECG Restitution Analysis and Machine Learning ...,Atrial fibrillation is the most frequent arrhy...,paroxysmal atrial fibrillation,0.391101,normal sinus-rhythm ECGs,0.316086,"[[Atrial Fibrillation, C0004238, Disease or Sy...","[[Atrial Fibrillation, C0004238, Disease or Sy...",NaN,NaN,NaN,"[[Normal sinus rhythm, C0232202, Finding], [El...","[[Electrocardiography, C1623258, Diagnostic Pr..."
3,35330920,Segmenting Thoracic Cavities with Neoplastic L...,Automatic segmentation of thoracic cavity stru...,neoplastic disease,0.441814,CT images,0.502925,"[[Neoplastic disease, C1882062, Neoplastic Pro...","[[Neoplastic disease, C1882062, Neoplastic Pro...",NaN,NaN,NaN,"[[X-Ray Computed Tomography, C0040405, Diagnos...","[[X-Ray Computed Tomography, C0040405, Diagnos..."
4,30814403,Combined machine learning and functional magne...,Hypoxia exposure during high-altitude expediti...,psychomotor impairment,0.347314,Rs-fMRI,0.341074,"[[Impaired, C0221099, Functional Concept]]",NaN,NaN,"[[Impaired, C0221099, Functional Concept]]",NaN,NaN,NaN
5,33075216,Simple risk score to screen for prediabetes: A...,The progression from prediabetes to typeÃÂ 2 ...,prediabetes,0.198258,Qatar Biobank cohort,0.448133,"[[Prediabetes syndrome, C0362046, Disease or S...","[[Prediabetes syndrome, C0362046, Disease or S...",NaN,NaN,NaN,"[[biorepository, C1711176, Manufactured Object...",NaN
6,34933537,The use and applicability of machine learning ...,Artificial intelligence (AI) is increasingly u...,benign prostatic enlargement,0.648665,Clinical data,0.333634,"[[benign, C0205183, Qualitative Concept], [Enl...","[[Enlargement (morphologic abnormality), C2711...",NaN,"[[Enlargement (morphologic abnormality), C2711...",NaN,"[[Clinical data, C1516606, Intellectual Product]]","[[Clinical data, C1516606, Intellectual Product]]"
7,34897147,Automation of a Rule-based Workflow to Estimat...,Myelination-related MR signal changes in white...,Sturge-Weber syndrome,0.564898,T1-weighted images (T1WIs) and T2-weighted ima...,0.187111,NaN,NaN,NaN,NaN,NaN,"[[Entire, C0439751, Quantitative Concept], [C1...","[[Diagnostic Imaging, C0011923, Diagnostic Pro..."
8,34176867,Deep Learning Algorithm to Detect Cardiac Sarc...,Because the early diagnosis of subclinical car...,cardiac sarcoidosis,0.436207,echocardiographic movies,0.345082,"[[Cardiac sarcoidosis, C0392077, Disease or Sy...","[[Cardiac sarcoidosis, C0392077, Disease or Sy...",NaN,NaN,NaN,"[[Echocardiography, C0013516, Diagnostic Proce...","[[Echocardiography, C0013516, Diagnostic Proce..."
9,25296360,Estimating the average length of hospitalizati...,Exposure to air pollutants is associated with ...,pneumonia,0.449299,air pollutants,0.167810,"[[Pneumonia, C0032285, Disease or Syndrome]]","[[Pneumonia, C0032285, Disease or Syndrome]]",NaN,NaN,NaN,"[[air pollutants, C0001869, Hazardous or Poiso...",NaN


In [ ]:
#load API responses from previously analysed data
#with open("disease_SNOMED", "r") as fp:
#    disease_SNOMED = json.load(fp)
#    
#print(disease_SNOMED[0])
#
#with open("disease_combined_SNOMED", "r") as fp:
#    disease_combined_SNOMED = json.load(fp)
#    
#print(disease_combined_SNOMED[0])
#
#with open("modality_SNOMED", "r") as fp:
#    modality_SNOMED = json.load(fp)
#    
#print(modality_SNOMED[0])

In [10]:
#convert CUIs to SNOMED codes with linked names
def SNOMED_convert(series):
    API_url = "https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{}/atoms?sabs=SNOMEDCT_US&returnIdType=code&ttys=PT&language=ENG&apiKey=d2d3d759-9022-4b37-92ba-bde3d35265ba"
    SNOMED_list = []
    failed_CUI = []
    fail_count = 0
    count = 0
    for item in series:
        count += 1
        if count%1000 == 0:
#        if count%100 == 0:
            print("Processed: "+str(count)+" records")
        item_list = []
        if str(item) != "nan":
            for x in item:
                try:
                    #post to API and create list of name and SNOMED code for each concept
                    time.sleep(0.15)
                    url = API_url.format(x[1])
                    data = urllib.request.urlopen(url)
                    decoded = data.read().decode('utf-8')
                    json_decoded = json.loads(decoded)
                    SNOMED_name = json_decoded["result"][0]["name"]
                    SNOMED_link = json_decoded["result"][0]["sourceConcept"]
                    SNOMED_code = SNOMED_link.split("/")[-1]
                    x_list = [SNOMED_name, SNOMED_code]
                    item_list.append(x_list)
                except:
                    if x not in failed_CUI:
                        fail_count += 1
                        failed_CUI.append(x)
        else:
            item_list = np.NaN
        if item_list == []:
            item_list = np.NaN
        SNOMED_list.append(item_list)
    return [SNOMED_list, failed_CUI]
        
disease_SNOMED = SNOMED_convert(disease_mod["discovered_disease"])
disease_combined_SNOMED = SNOMED_convert(disease_mod["discovered_disease_combined"])
modality_SNOMED = SNOMED_convert(disease_mod["discovered_modality"])

Processed: 1000 records
Processed: 2000 records
Processed: 3000 records
Processed: 4000 records
Processed: 5000 records
Processed: 6000 records
Processed: 7000 records
Processed: 8000 records
Processed: 9000 records
Processed: 10000 records
Processed: 11000 records
Processed: 12000 records
Processed: 13000 records
Processed: 14000 records
Processed: 15000 records
Processed: 16000 records
Processed: 17000 records
Processed: 18000 records
Processed: 19000 records
Processed: 20000 records
Processed: 21000 records
Processed: 22000 records
Processed: 23000 records
Processed: 24000 records
Processed: 25000 records
Processed: 26000 records
Processed: 27000 records
Processed: 28000 records
Processed: 1000 records
Processed: 2000 records
Processed: 3000 records
Processed: 4000 records
Processed: 5000 records
Processed: 6000 records
Processed: 7000 records
Processed: 8000 records
Processed: 9000 records
Processed: 10000 records
Processed: 11000 records
Processed: 12000 records
Processed: 13000 r

In [52]:
#save API responses to file for later use to avoid lengthly analysis process
with open("disease_SNOMED", "w") as fp:
    json.dump(disease_SNOMED, fp)
    
with open("disease_combined_SNOMED", "w") as fp:
    json.dump(disease_combined_SNOMED, fp)
    
with open("modality_SNOMED", "w") as fp:
    json.dump(modality_SNOMED, fp)

In [37]:
disease_mod["disease_SNOMED"] = pd.Series(disease_SNOMED[0])
disease_mod["disease_combined_SNOMED"] = pd.Series(disease_combined_SNOMED[0])
disease_mod["modality_SNOMED"] = pd.Series(modality_SNOMED[0])
disease_mod.to_csv("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Health AI Tracking\\disease_modality\\disease_modality_SNOMED.csv")
disease_mod.info()
disease_mod.head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28703 entries, 0 to 28702
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   pmid                         28703 non-null  int64  
 1   title                        28702 non-null  object 
 2   abstract                     28703 non-null  object 
 3   disease_answer               28703 non-null  object 
 4   disease_score                28703 non-null  float64
 5   modality_answer              28703 non-null  object 
 6   modality_score               28703 non-null  float64
 7   all_discovered_disease       27046 non-null  object 
 8   discovered_disease           21754 non-null  object 
 9   discovered_location          3105 non-null   object 
 10  discovered_finding           3330 non-null   object 
 11  discovered_disease_combined  827 non-null    object 
 12  all_discovered_modality      26347 non-null  object 
 13  discovered_modal

,pmid,title,abstract,disease_answer,disease_score,modality_answer,modality_score,all_discovered_disease,discovered_disease,discovered_location,discovered_finding,discovered_disease_combined,all_discovered_modality,discovered_modality,disease_SNOMED,disease_combined_SNOMED,modality_SNOMED
0,35309968,mTeeth: Identifying Brushing Teeth Surfaces Us...,Ensuring that all the teeth surfaces are adequ...,oral diseases,0.304024,inertial sensors,0.559315,"[[Disease, Mouth, C0026636, Disease or Syndrome]]","[[Disease, Mouth, C0026636, Disease or Syndrome]]",NaN,NaN,NaN,NaN,NaN,"[[Disease of mouth, 118938008]]",NaN,NaN
1,35330785,Development of a Machine Learning Algorithm fo...,Reverse total shoulder arthroplasty (rTSA) off...,Reverse total shoulder arthroplasty,0.377661,Office of Statewide Health Planning and Develo...,0.363589,"[[Arthroplasties, Shoulder Replacement, C01866...","[[Arthroplasties, Shoulder Replacement, C01866...",NaN,NaN,NaN,"[[Intellectual Product, C0282574, Intellectual...","[[Intellectual Product, C0282574, Intellectual...","[[Total shoulder replacement, 42262007]]",NaN,NaN
2,35330977,ECG Restitution Analysis and Machine Learning ...,Atrial fibrillation is the most frequent arrhy...,paroxysmal atrial fibrillation,0.391101,normal sinus-rhythm ECGs,0.316086,"[[Atrial Fibrillation, C0004238, Disease or Sy...","[[Atrial Fibrillation, C0004238, Disease or Sy...",NaN,NaN,NaN,"[[Normal sinus rhythm, C0232202, Finding], [El...","[[Electrocardiography, C1623258, Diagnostic Pr...","[[Atrial fibrillation, 49436004]]",NaN,"[[Electrocardiographic procedure, 29303009]]"
3,35330920,Segmenting Thoracic Cavities with Neoplastic L...,Automatic segmentation of thoracic cavity stru...,neoplastic disease,0.441814,CT images,0.502925,"[[Neoplastic disease, C1882062, Neoplastic Pro...","[[Neoplastic disease, C1882062, Neoplastic Pro...",NaN,NaN,NaN,"[[X-Ray Computed Tomography, C0040405, Diagnos...","[[X-Ray Computed Tomography, C0040405, Diagnos...","[[Neoplastic disease, 55342001]]",NaN,"[[Computed tomography, 77477000]]"
4,30814403,Combined machine learning and functional magne...,Hypoxia exposure during high-altitude expediti...,psychomotor impairment,0.347314,Rs-fMRI,0.341074,"[[Impaired, C0221099, Functional Concept]]",NaN,NaN,"[[Impaired, C0221099, Functional Concept]]",NaN,NaN,NaN,NaN,NaN,NaN
5,33075216,Simple risk score to screen for prediabetes: A...,The progression from prediabetes to typeÃÂ 2 ...,prediabetes,0.198258,Qatar Biobank cohort,0.448133,"[[Prediabetes syndrome, C0362046, Disease or S...","[[Prediabetes syndrome, C0362046, Disease or S...",NaN,NaN,NaN,"[[biorepository, C1711176, Manufactured Object...",NaN,"[[Prediabetes, 714628002]]",NaN,NaN
6,34933537,The use and applicability of machine learning ...,Artificial intelligence (AI) is increasingly u...,benign prostatic enlargement,0.648665,Clinical data,0.333634,"[[benign, C0205183, Qualitative Concept], [Enl...","[[Enlargement (morphologic abnormality), C2711...",NaN,"[[Enlargement (morphologic abnormality), C2711...",NaN,"[[Clinical data, C1516606, Intellectual Product]]","[[Clinical data, C1516606, Intellectual Product]]","[[Enlargement, 442021009]]",NaN,NaN
7,34897147,Automation of a Rule-based Workflow to Estimat...,Myelination-related MR signal changes in white...,Sturge-Weber syndrome,0.564898,T1-weighted images (T1WIs) and T2-weighted ima...,0.187111,NaN,NaN,NaN,NaN,NaN,"[[Entire, C0439751, Quantitative Concept], [C1...","[[Diagnostic Imaging, C0011923, Diagnostic Pro...",NaN,NaN,"[[Imaging, 363679005], [Laboratory procedure, ..."
8,34176867,Deep Learning Algorithm to Detect Cardiac Sarc...,Because the early diagnosis of subclinical car...,cardiac sarcoidosis,0.436207,echocardiographic movies,0.345082,"[[Cardiac sarcoidosis, C0392077, Disease or Sy...","[[Cardiac sarcoidosis, C0392077, Disease or Sy...",NaN,NaN,NaN,"[[Echocardiography, C0013516, Diagnostic Proce...","[[Echocardiography, C0013516, Diagnostic Proce...","[[Cardiac sarcoidosis, 75403004]]",NaN,"[[Ec

In [12]:
#get list of failed CUIs
failed_CUI_disease = disease_SNOMED[1]
failed_CUI_combined = disease_combined_SNOMED[1]
failed_CUI_modality = modality_SNOMED[1]

failed_CUI_total = [x for x in failed_CUI_disease] + [x for x in failed_CUI_combined if x not in failed_CUI_disease] + [x for x in failed_CUI_modality if x not in failed_CUI_disease]
print(len(failed_CUI_total))
print(failed_CUI_total)

1371
[['Metastasis', 'C4255448', 'Pathologic Function'], ['Colorectal Cancer', 'C1527249', 'Neoplastic Process'], ['Rectal Carcinoma', 'C0007113', 'Neoplastic Process'], ['Stroke/Brain Attack', 'C3844825', 'Disease or Syndrome'], ['Diabetes', 'C0011847', 'Disease or Syndrome'], ['Peripheral Neuropathy', 'C0031117', 'Disease or Syndrome'], ['Coronary heart disease', 'C0010068', 'Disease or Syndrome'], ['cancers cell neuroendocrine', 'C0746852', 'Neoplastic Process'], ['Ovarian Carcinoma', 'C0029925', 'Neoplastic Process'], ['Carcinoma breast stage IV', 'C0278488', 'Neoplastic Process'], ['Decision', 'C0679006', 'Mental Process'], ['Impulsive Behavior', 'C0021125', 'Mental or Behavioral Dysfunction'], ['Autism Spectrum Disorders', 'C1510586', 'Mental or Behavioral Dysfunction'], ['cervical cancer', 'C4048328', 'Neoplastic Process'], ['Carcinoma of lung', 'C0684249', 'Neoplastic Process'], ['ST segment elevation myocardial infarction', 'C1536220', 'Disease or Syndrome'], ['Minimal Access 